In [ ]:
import pandas as pd
import datetime
df = pd.read_csv('03_Library Systembook.csv')

# column names
print("\nColumn names:")
print(df.columns.tolist())

# drop empty rows and columns and useless records 
df = df.dropna(how='all')
df = df.dropna(axis=1, how = 'all')
df = df.dropna(subset=['Customer ID'])
# Convert ID to INT
df['Id'] = df['Id'].astype(int) 
df['Customer ID'] = df['Customer ID'].astype(int) 
# trim spaces
df = df.apply(lambda x:x.str.strip().str.replace('"','',regex=False) if x.dtype == "object" else x)
# Properly case the book titles
df['Books'] = df['Books'].str.title()
# convert Weeks to days
df['Days allowed to borrow'] = df['Days allowed to borrow'].str.replace('2 weeks','14',regex=False)
#Custom Replace function
def clean_text(text_series):
    """
    Apply multiple text replacements to a column.
    Add or modify replacements in the list below.
    """
    replacements = [
        ('"', ''),                          # Remove Quotes 
        ('32/05/2023', '31/05/2023'),       # Correct Date
        ('Of The Kind', 'Of The King')     # Correct Title      
    ]
    
    for old, new in replacements:
        text_series = text_series.str.replace(old, new, regex=False)
    
    return text_series

def days_on_loan(bookout,bookin):
    """
    Calculate the time the book was on loan
    """
    on_loan = (bookin - bookout).dt.days   
    return on_loan

df = df.apply(lambda x:clean_text(x) if x.dtype == "object" else x)

#Convert to Datetime
df['Book checkout'] = pd.to_datetime(df['Book checkout'],dayfirst=True)
df['Book Returned'] = pd.to_datetime(df['Book Returned'],dayfirst=True)
df['Days On Loan'] = days_on_loan(df['Book checkout'], df['Book Returned'])

print ("Added Loan Duration")
print(df)


Column names:
['Id', 'Books', 'Book checkout', 'Book Returned', 'Days allowed to borrow', 'Customer ID']
Added Loan Duration

Cleansed Dataframe:
    Id                                     Books Book checkout Book Returned  \
0    1                        Catcher In The Rye    2023-02-20    2023-02-25   
1    2          Lord Of The Rings The Two Towers    2023-03-24    2023-03-21   
2    3  Lord Of The Rings The Return Of The King    2023-03-29    2023-03-25   
3    4                                The Hobbit    2023-04-02    2023-03-25   
4    5                                      Dune    2023-04-02    2023-03-25   
5    6                              Little Women    2023-04-02    2023-05-01   
6    7                                        It    2063-04-10    2023-04-03   
7    8                                    Misery    2023-04-15    2023-04-03   
8    9                                  Catch 22    2023-04-15    2023-04-16   
9   10                               Animal Farm    2

import pandas as pd

df = pd.read_csv('03_Library SystemCustomers.csv')

df = df.dropna(subset=['Customer ID'])
df['Customer ID'] = df['Customer ID'].astype(int) 
print("\nCleansed Dataframe:")
print(df)


from sqlalchemy import create_engine, text ,inspect
import pandas as pd

server = 'localhost'
database_name = 'M5-Library'
table1_name = 'Books'
table2_name = 'Customers'

master_connection_string = f'mssql+pyodbc://{server}/master?driver=ODBC+Driver+17+for+SQL+Server&trusted_connections=yes'
master_engine = create_engine(master_connection_string)

print(f"Checking For Db '{database_name}'...")
with master_engine.connect() as conn:
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    result = conn.execute(text(f"SELECT database_id FROM sys.databases WHERE name ='{database_name}'"))
    db_exists = result.fetchone() is not None

    if db_exists:
        print(f"Database '{database_name}' already exists")
    else:
        print(f"Database '{database_name}' not found. Creating ...")
        conn.execute(text(f"CREATE DATABASE [{database_name}]"))
        print(f"Database '{database_name}' created sucessfully.")


